# Content based Recommendation system

Importing the dependancines

In [1]:
# !pip install cdifflib

In [2]:
import numpy as np
import pandas as pd
import difflib # to give recommendation based on the user input even if the spelling is wrong it will find the closer (similar) movie
from sklearn.feature_extraction.text import TfidfVectorizer # to convert textual data into metrics (numeric) form
from sklearn.metrics.pairwise import cosine_similarity # to find the most simmilar movie to the user input movie

In [3]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",300)
pd.set_option("display.max_colwidth",100)


Data Collection and Pre-processing

In [4]:
# loading the data from csv file to the pandas dataframe
movies_data = pd.read_csv(r"E:\Movie_recommendation_system\movies.csv")

In [5]:
# printing the first 5 rows of data
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony society,en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'department': 'Editing', 'job': 'Editor', 'credit_id...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trading company love of one's life,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\u00e5rd Chow Yun-fat,"[{'name': 'Dariusz Wolski', 'gender': 2, 'department': 'Camera', 'job': 'Director of Photography...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux Ralph Fiennes Monica Bellucci,"[{'name': 'Thomas Newman', 'gender': 2, 'department': 'Sound', 'job': 'Original Music Composer',...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret identity burglar,en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne Hathaway Tom Hardy,"[{'name': 'Hans Zimmer', 'gender': 2, 'department': 'Sound', 'job': 'Original Music Composer', '...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel princess,en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"

In [6]:
movies_data["original_title"]

0                                                                                       Avatar
1                                                     Pirates of the Caribbean: At World's End
2                                                                                      Spectre
3                                                                        The Dark Knight Rises
4                                                                                  John Carter
5                                                                                 Spider-Man 3
6                                                                                      Tangled
7                                                                      Avengers: Age of Ultron
8                                                       Harry Potter and the Half-Blood Prince
9                                                           Batman v Superman: Dawn of Justice
10                                                

In [7]:
# number of rows and columns in dataframe
movies_data.shape

(4803, 24)

In [8]:
# printing the columns list
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [9]:
# selecting the relevent feature for recommendation
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [10]:
# replacing the null values with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [11]:
# combining all 5 selected features
combined_features = movies_data['genres']+ " "+ movies_data['keywords']+ " "+movies_data['tagline']+ " "+movies_data['cast']+ " "+movies_data['director']

In [12]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction culture clash future space war space colony society Ent...
1       Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's...
2       Action Adventure Crime spy based on novel secret agent sequel mi6 A Plan No One Escapes Daniel C...
3       Action Crime Drama Thriller dc comics crime fighter terrorist secret identity burglar The Legend...
4       Action Adventure Science Fiction based on novel mars medallion space travel princess Lost in our...
5       Fantasy Action Adventure dual identity amnesia sandstorm love of one's life forgiveness The batt...
6       Animation Family hostage magic horse fairy tale musical They're taking adventure to new lengths....
7       Action Adventure Science Fiction marvel comic sequel superhero based on comic book vision A New ...
8       Adventure Fantasy Family witch magic broom school of witchcraft wizardry Dark Secrets Revealed D...
9       Action Adventure Fan

In [13]:
# converting text data to feature vector
vectorizer = TfidfVectorizer()

In [14]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [15]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Cosine Similarity

In [16]:
# getting similarity score using cosine similarity
similarity = cosine_similarity(feature_vectors) # how one movie is similar to other movie

In [17]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [18]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from user

In [19]:
# getting the movie name from user
movie_name = input('Enter your favourite movie name : ')

Enter your favourite movie name : spider man


In [20]:
# creating a list with all the movie title name
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [21]:
# finding the close match to the movie given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Spider-Man', 'Inside Man', 'Superman']


In [22]:
# to get the first match to the movie name
close_match = find_close_match[0]
print(close_match)

Spider-Man


In [23]:
# finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

159


In [24]:
# getting the list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.05803347266824191), (1, 0.028510860472594192), (2, 0.027527242422615502), (3, 0.006457320533567938), (4, 0.07910312941508459), (5, 0.3188331558421017), (6, 0.0), (7, 0.005837358717098843), (8, 0.01984856933034166), (9, 0.017094559158687336), (10, 0.03289500092097852), (11, 0.00513746235036844), (12, 0.017699463936031615), (13, 0.016312073634578168), (14, 0.017221598804829895), (15, 0.011028690492438575), (16, 0.0055350569108376544), (17, 0.016931592006051974), (18, 0.005525360643470355), (19, 0.021680168394872627), (20, 0.04593061379094501), (21, 0.004853496672605501), (22, 0.017356132823837148), (23, 0.028252755876041348), (24, 0.01268769144627989), (25, 0.011992878106901545), (26, 0.006024661849598387), (27, 0.005268179012698267), (28, 0.005783838859717594), (29, 0.028149881853568752), (30, 0.3179190198222972), (31, 0.03510954856669071), (32, 0.021374196216168765), (33, 0.005801089457404634), (34, 0.0), (35, 0.006089692551768507), (36, 0.005487298957648992), (37, 0.13254335177

In [25]:
len(similarity_score)

4803

In [26]:
# sorting the movies based on the similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x: x[1], reverse = True)
print(sorted_similar_movies)

[(159, 1.0000000000000004), (5, 0.3188331558421017), (30, 0.3179190198222972), (1559, 0.1828131277152563), (382, 0.16501718739122473), (3575, 0.16167041055131565), (2361, 0.13913552967690979), (37, 0.13254335177245744), (1364, 0.1315753447298594), (1193, 0.1282353330125354), (1793, 0.12355727076460596), (328, 0.12188313538843792), (677, 0.11987005023906432), (1796, 0.11833294042448737), (1523, 0.11678331012908381), (1598, 0.114546607093097), (4441, 0.11278570204538363), (2529, 0.10580630139750406), (4427, 0.10339492817261244), (976, 0.10254994461653799), (2157, 0.10224484429143274), (978, 0.1010166095053646), (2369, 0.09964302137705913), (1435, 0.09844672492278542), (448, 0.09796336022022192), (1533, 0.09756374595438326), (2060, 0.09746246263620428), (2201, 0.09721199893417153), (2980, 0.09682299898140828), (2763, 0.09599045327407793), (3077, 0.09545910274922952), (3155, 0.09513620988808803), (3046, 0.09500017389530692), (2592, 0.09458996594241367), (1239, 0.09280098679642097), (3188, 

In [27]:
# print the similar movies based on the index
print('movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if (i<21):
        print(i, " - ", title_from_index)
        i += 1

movies suggested for you : 

1  -  Spider-Man
2  -  Spider-Man 3
3  -  Spider-Man 2
4  -  The Notebook
5  -  Seabiscuit
6  -  Clerks II
7  -  The Ice Storm
8  -  Oz: The Great and Powerful
9  -  Horrible Bosses
10  -  The Count of Monte Cristo
11  -  In Good Company
12  -  Finding Nemo
13  -  Clear and Present Danger
14  -  Brothers
15  -  The Good German
16  -  Drag Me to Hell
17  -  Bambi
18  -  The Queen
19  -  Charly
20  -  Escape from L.A.


# Movie Recommendation System

In [33]:
movie_name = input('Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x: x[1], reverse = True)

print('movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if (i<21):
        print(i, " - ", title_from_index)
        i += 1

Enter your favourite movie name : NO BOUDDY DOES IT BETTER


IndexError: list index out of range

In [29]:
# NO BOUDDY DOES IT BETTER

In [30]:
import pickle

In [31]:
pickle.dump(movies_data, open('movies_data.sav', 'wb'))
pickle.dump(similarity, open('similarity.sav', 'wb'))